In [ ]:
from PyPDF2 import PdfReader
import fitz
import pdfplumber
import pandas, numpy

In [ ]:
file_path = r"/content/drive/MyDrive/data-sci/data-projects/pdf-extractor/data/"

In [ ]:
file_name = r"scanned-pdf/fact_sheet_all_data.pdf"

final_path = file_path + file_name
print(final_path)

/content/drive/MyDrive/data-sci/data-projects/pdf-extractor/data/scanned-pdf/fact_sheet_all_data.pdf


In [ ]:
def get_single_page(final_path):
  pdf_doc = fitz.open(final_path)
  page = pdf_doc[2]
  page = get_page_content(page)
  print(page)

get_single_page(final_path)

In [ ]:
# def get_text_from_pdf(final_path):
#     text = ""
#     with open(file_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         num_pages = len(reader.pages)
#         print(num_pages);
#         for page_num in range(num_pages):
#             page = reader.pages[page_num]
#             text += page.extract_text()
#     #print(text)
#     return text


# document -> page -> block -> line -> span

In [ ]:
def parse_document(file_path):
    document = {}

    pdf_doc = fitz.open(file_path)
    total_pages = len(pdf_doc)

    for page_number in range(total_pages):
        page = pdf_doc[page_number]
        parsed_content = get_page_content(page)
        document[f"page{page_number + 1}"] = parsed_content

    return document


def get_page_content(page):
    block_content = {"header": [], "content": {}}
    block_size = {}

    # Obtain each block in the paragraph
    blocks = page.get_text("dict")["blocks"]
    total_blocks = len(blocks)

    for block_index in range(total_blocks):
        block = blocks[block_index]
        text_font_size = []

        if "lines" in block:
            for line in block["lines"]:
                for span in line["spans"]:
                    text_font_size.append((span["text"], span["size"], span["font"]))

        # Organize content based on font size
        header_text = []
        paragraph_text = []
        for text, size, font in text_font_size:
            if size == 12:
                paragraph_text.append(text)
            else:
                header_text.append(text)

        # Process header and content separately
        if header_text:
            block_content["header"].append(" ".join(header_text))  # Combine header text
        if paragraph_text:
            block_content["content"][f"p{block_index + 1}"] = " ".join(paragraph_text)  # Combine paragraph text

    return block_content


In [ ]:
final_data = get_content(final_path)

print(final_data["page1"])

{'header': ['6', 'The Macro picture', 'November 2024 October 2024', 'Index Name Return in % As on', '29-Nov-24 31-Oct-24 30-Nov-23 1 Month 1 Year Index Name Return in % As on', '29-Nov-24 31-Oct-24 30-Nov-23 1 Month 1 Year', 'www.tatamutualfund.com TATA MUTUAL FUND', 'MARKET OUTLOOK', 'Equity market', '', 'Disclaimer:  The views expressed are in no way trying to predict the markets or to time them. The views expressed are for information purpose only and do not construe to be any investment, legal or taxation advice.  Please consult your Financial/Investment Adviser before investing.  The views expressed may not reflect in the scheme portfolios of Tata Mutual Fund. This note has been prepared using information believed to be  accurate at the time of its use.', 'BSE-30 and Nifty-50 indices consolidated for the month of November 2024 post a sharp correction in the previous month. Mid-cap and Small Cap indices also performed inline with large caps. On the sectoral Capital Goods, IT, Banks

In [ ]:
def get_content(file_path):
    document = {}
    pdf_doc = fitz.open(file_path)
    total_pages = len(pdf_doc)

    for page_number in range(total_pages):
        page = pdf_doc[page_number]
        page_content = parse_page(page)
        document[f"page{page_number + 1}"] = page_content

    return document


def parse_page(page):
    block_data = {"header": [], "content": {}}
    blocks = page.get_text("dict")["blocks"]

    # Identify most common font size (likely body text)
    font_sizes = []
    for block in blocks:
        if "lines" in block:
            for line in block["lines"]:
                for span in line["spans"]:
                    font_sizes.append(span["size"])
    most_common_font_size = max(set(font_sizes), key=font_sizes.count)

    # Process each block
    for block_index, block in enumerate(blocks):
        if "lines" in block:
            block_text, is_header = process_block(block, most_common_font_size)
            if is_header:
                block_data["header"].append(block_text)
            else:
                block_data["content"][f"p{block_index + 1}"] = block_text

    return block_data


def process_block(block, common_font_size):
    text_content = []
    is_header = False

    for line in block["lines"]:
        for span in line["spans"]:
            text_content.append(span["text"])
            # Determine if it's a header
            if span["size"] > common_font_size:
                is_header = True

    return " ".join(text_content).strip(), is_header


In [ ]:
import fitz  # PyMuPDF
import pandas as pd

def extract_pdf_data(pdf_path):
    pdf_doc = fitz.open(pdf_path)
    data = {"header": [], "content": {}, "footer": [], "tables": []}
    table_count = 0

    for page_num, page in enumerate(pdf_doc):
        blocks = page.get_text("dict")["blocks"]

        page_data = {"header": [], "content": [], "footer": []}
        for block in blocks:
            if "lines" in block:
                text = []
                for line in block["lines"]:
                    for span in line["spans"]:
                        text.append(span["text"].strip())

                # Join the extracted text for the block
                block_text = " ".join(text).strip()

                # Classify text based on position and font size
                if block["bbox"][1] < 100:  # Top of the page
                    page_data["header"].append(block_text)
                elif block["bbox"][1] > (page.rect.height - 100):  # Bottom of the page
                    page_data["footer"].append(block_text)
                else:
                    page_data["content"].append(block_text)

                # Detect potential tables
                if "lines" in block and len(block["lines"]) > 1 and any("\t" in span["text"] for line in block["lines"] for span in line["spans"]):
                    table_count += 1
                    table_data = extract_table(block)
                    data["tables"].append((f"table{table_count}.csv", table_data))

        # Consolidate page data
        data["header"].extend(page_data["header"])
        data["footer"].extend(page_data["footer"])
        data["content"][f"page{page_num + 1}"] = page_data["content"]

    # Save tables to CSV files
    save_tables(data["tables"])

    return data


def extract_table(block):
    """Extract table-like data from a block."""
    table_rows = []
    for line in block["lines"]:
        row = []
        for span in line["spans"]:
            row.append(span["text"].strip())
        table_rows.append(row)
    return table_rows


def save_tables(tables):
    """Save table data to CSV files."""
    for table_name, table_data in tables:
        df = pd.DataFrame(table_data)
        df.to_csv(table_name, index=False, header=False)


def create_readme(data):
    """Create a README file explaining the structure of the extracted data."""
    readme_text = """
    This README file explains the structure of the extracted data.

    1. Headers:
       - All headers extracted from the top of the pages are stored in the "header" section.

    2. Content:
       - Each page's content is stored as a list in the "content" section.
       - The key for each page's content is "page1", "page2", etc.

    3. Footers:
       - All footers extracted from the bottom of the pages are stored in the "footer" section.

    4. Tables:
       - Each table is saved as a separate CSV file (e.g., table1.csv, table2.csv).
       - The tables are stored in the "tables" section as a list of file names.

    5. Example:
       {
           "header": ["Main Header", "Sub Header"],
           "content": {
               "page1": ["Content line 1", "Content line 2"],
               "page2": ["More content here."]
           },
           "footer": ["Page 1 footer text", "Page 2 footer text"],
           "tables": ["table1.csv", "table2.csv"]
       }
    """
    with open("README.txt", "w") as readme_file:
        readme_file.write(readme_text)


# Process the uploaded PDF
pdf_path = "/mnt/data/fact_sheet_all_data.pdf"
extracted_data = extract_pdf_data(final_path)

# Create a README
create_readme(extracted_data)


In [ ]:
extracted_data